### TABLE CREATION

In [ ]:
from CreateTable import create_df
import pickle
import pandas as pd

# data_test = create_df("DamasconeC/together/*_CPXE_*.csv", symm_v = True, angle=-1, offset=(52, 51))  # , symmetric = True
# data_train = create_df("DamasconeA/data2/*_CPXE_*.csv", offset=(49, 51), angle=1)
# data_train = create_df("DamasconeB/together/*_CPXE_*.csv")
data_train = pd.read_pickle("DamasconeA2.pkl")

data_validation = pd.read_pickle("DamasconeA.pkl")
# data_train2 = data_train2.drop(columns=["Row"])

data_test = pd.read_pickle("DamasconeB.pkl")
# data_test = pd.read_pickle("C_norm_ss.pkl")

# to be handled properly TODO
data_train = data_train.dropna()
data_test = data_test.dropna()
data_validation = data_validation.dropna()


In [ ]:
import numpy as np

data_train['stiffness_to_relaxation'] = data_train['Stiffness'] / data_train['force_relaxation']
data_train['oscillation_to_max_force'] = data_train['force_oscillation'] / data_train['force_max']

data_test['stiffness_to_relaxation'] = data_test['Stiffness'] / data_test['force_relaxation']
data_test['oscillation_to_max_force'] = data_test['force_oscillation'] / data_test['force_max']

data_validation['stiffness_to_relaxation'] = data_validation['Stiffness'] / data_validation['force_relaxation']
data_validation['oscillation_to_max_force'] = data_validation['force_oscillation'] / data_validation['force_max']


In [ ]:
data_test.to_pickle('B_norm_ss.pkl')

In [ ]:
from helper import feature_list2

# Create dataset with selected features (no posx, posy)
# feature_list_time = [ 'Stiffness', 'Upstroke', 'P_ss']
# feature_list_hysteresis = ['peak_position','loading_energy']
# feature_list = feature_list_time + feature_list_hysteresis
# feature_list = feature_list2
#
# to_remove = ['segment2_slope', 'segment2_force_std',
#              'segment2_skew', 'segment3_skew', 'Downstroke', 'poly3_coef0',
#              'poly5_coef0', 'poly5_coef1', 'poly5_coef2', 'poly5_coef3', 'poly5_coef4', 'Entropy', "Tau", 'P_ss',
#              'offset', 'force_relaxation', 'force_ratio_75_25', 'hysteresis_area'
#              ]
#
# feature_list = [x for x in feature_list2 + ['stiffness_to_relaxation', 'oscillation_to_max_force'] if x not in to_remove]
# feature_list = ['loading_unloading_area_ratio', 'peak_width', 'stiffness_to_relaxation', 'Stiffness']
feature_list = ["Stiffness", "Upstroke", "Downstroke", "P_ss", "peak_position", "loading_energy"]

### PLOTS

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import resample


def plot_aligned_samples(df, label_column, force_column, pos_column, threshold=0.1, max_samples_per_label=5):
    unique_labels = (0, 1, 2, 3, 4, 5)

    fig, axes = plt.subplots(1, 1, figsize=(8, 8))

    used_labels = set()  # Track labels for legend

    for label in unique_labels:
        # Get all samples for this label
        label_df = df[df[label_column] == label]

        # Downsample if too many samples
        if len(label_df) > max_samples_per_label:
            label_df = resample(label_df, n_samples=max_samples_per_label, random_state=42)

        for _, sample in label_df.iterrows():
            # Extract force and position
            force = np.array(sample[force_column])
            position = np.array(sample[pos_column])

            # Find alignment point
            force_above_threshold = force > threshold
            if np.any(force_above_threshold):
                start_idx = np.where(force_above_threshold)[0][0]

                # Label only the first occurrence of each label
                label_text = f'Label {label}' if label not in used_labels else None
                used_labels.add(label)

                # Plot aligned data
                axes.plot(position - position[start_idx], force, label=label_text, alpha=0.7)

    # Set plot properties
    axes.set_xlabel('Position')
    axes.set_xlim(left=-0.001)
    axes.set_ylabel('Force')
    axes.set_title('Force over Time (Aligned)')
    axes.legend()
    axes.grid()

    plt.suptitle('Aligned Sample Plot')
    plt.tight_layout()
    plt.show()


# Example usage
plot_aligned_samples(data_train, label_column='label', force_column='Fi', pos_column='Pi', threshold=0.05,
                     max_samples_per_label=1)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def plot_aligned_samples(df, label_column, force_column, pos_column, time_column, n=1, threshold=0.1):
    unique_labels = df[label_column].unique()  # Get unique labels
    unique_labels = (0, 1, 2, 3, 4, 5)

    for i in range(n):  # Repeat n times
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # 1 row, 2 columns

        for label in unique_labels:
            # Randomly select one sample per label
            sample = df[df[label_column] == label].sample(1, random_state=np.random.randint(0, 10000))

            # Extract force, position, and time
            force = np.array(sample[force_column].tolist()[0])
            position = np.array(sample[pos_column].tolist()[0])
            time = np.array(sample[time_column].tolist()[0])

            # Find the first index where force rises above the threshold
            start_idx = np.where(force > threshold)[0][0]

            # Align the time axis so that this point is at t = 0
            time_aligned = time - time[start_idx]

            # Plot force vs. time (left subplot)
            axes[0].plot(time_aligned, force, label=f'Label {label}')
            axes[0].set_xlabel('Time (Aligned)')
            axes[0].set_ylabel('Force')
            axes[0].set_title('Force over Time (Aligned)')
            axes[0].legend()
            axes[0].grid()

            # Plot position vs. time (right subplot)
            axes[1].plot(time_aligned, position, label=f'Label {label}')
            axes[1].set_xlabel('Time (Aligned)')
            axes[1].set_ylabel('Position')
            axes[1].set_title('Position over Time (Aligned)')
            axes[1].legend()
            axes[1].grid()

        plt.suptitle(f'Aligned Sample Plot {i + 1}')
        plt.tight_layout()
        plt.show()


# Example usage
plot_aligned_samples(data_train, label_column='label', force_column='Fi', pos_column='Pi', time_column='Timei', n=2,
                     threshold=0.05)

### SPATIAL SMOOTHING

In [ ]:
from scipy.interpolate import NearestNDInterpolator
from scipy.ndimage import median_filter, gaussian_filter
import numpy as np
import pandas as pd
from helper import *

smoothing_config = {
    'Entropy': 'gaussian',
}  # Dictionary 'feature-name' : type of smoothing (set if you want to test other method of smoothing instead of median)


def apply_smoothing(grid_z, method='median'):
    if method == 'median':
        return median_filter(grid_z, size=3, mode='reflect')
    elif method == 'gaussian':
        return gaussian_filter(grid_z, sigma=2)
    elif method == 'diffusion':
        # Simple anisotropic diffusion
        def diffusion_step(img, kappa=50):
            # Compute image gradients
            dy, dx = np.gradient(img)

            # Compute diffusion coefficients
            diff_coef_x = 1 / (1 + (dx / kappa) ** 2)
            diff_coef_y = 1 / (1 + (dy / kappa) ** 2)

            # Compute diffusion
            diff_x = np.zeros_like(img)
            diff_y = np.zeros_like(img)

            diff_x[1:-1, 1:-1] = diff_coef_x[1:-1, 1:-1] * (img[1:-1, 2:] - img[1:-1, 1:-1])
            diff_y[1:-1, 1:-1] = diff_coef_y[1:-1, 1:-1] * (img[2:, 1:-1] - img[1:-1, 1:-1])

            return img + 0.25 * (diff_x + diff_y)

        # Apply multiple diffusion steps
        iterations = 20
        img = grid_z.copy()
        for i in range(iterations):
            img = diffusion_step(img)
        return img

    else:
        raise ValueError(f"Smoothing method not implemented: {method}")


def get_grid_bounds(df):
    x_min, x_max = int(df["posx"].min()), int(df["posx"].max())
    y_min, y_max = int(df["posy"].min()), int(df["posy"].max())
    grid_shape = (y_max - y_min + 1, x_max - x_min + 1)
    return x_min, y_min, grid_shape


# Smoothing function for a single dataframe
def smooth_subset(subset_df, x_min, y_min, grid_shape):
    smoothed_subset = pd.DataFrame(index=subset_df.index)

    for feature in feature_list:
        # Initialize grid with NaNs
        grid_z = np.full(grid_shape, np.nan, dtype=np.float32)

        # Map each data point to the grid
        for _, row in subset_df.iterrows():
            x_idx = int(row["posx"]) - x_min
            y_idx = int(row["posy"]) - y_min
            grid_z[y_idx, x_idx] = row[feature]

        # Interpolate missing values
        yy, xx = np.indices(grid_z.shape)
        valid_mask = ~np.isnan(grid_z)

        if np.any(~valid_mask):
            interpolator = NearestNDInterpolator(
                np.column_stack((yy[valid_mask], xx[valid_mask])),
                grid_z[valid_mask]
            )
            grid_z = interpolator(yy, xx)

        # Apply feature-specific smoothing
        methods = smoothing_config.get(feature,
                                       'median')  # Get the designed methods, if None select defaul 'median' method
        if isinstance(methods, list):
            for method in methods:
                grid_z = apply_smoothing(grid_z, method=method)
        else:
            grid_z = apply_smoothing(grid_z, method=methods)

        # Map smoothed grid back to DataFrame
        smoothed_subset[feature] = [
            grid_z[int(row["posy"]) - y_min, int(row["posx"]) - x_min]
            for _, row in subset_df.iterrows()
        ]

    # Add back metadata columns
    smoothed_subset[['label', 'posx', 'posy']] = subset_df[['label', 'posx', 'posy']]
    return smoothed_subset


if smoothing_config is None:
    smoothing_config = {feature: 'median' for feature in feature_list}

# Compute grid bounds and smooth each subset
test_x_min, test_y_min, test_grid_shape = get_grid_bounds(data_test)
train_x_min, train_y_min, train_grid_shape = get_grid_bounds(data_train)

# Smooth each subset SEPARATELY
smoothed_test = smooth_subset(data_test, test_x_min, test_y_min, test_grid_shape)
smoothed_train = smooth_subset(data_train, train_x_min, train_y_min, train_grid_shape)
smoothed_validation = smooth_subset(data_validation, train_x_min, train_y_min, train_grid_shape)

smoothed_df = pd.concat([smoothed_test, smoothed_train],
                        ignore_index=True)  # For visualization in the next sections only

### DB SPLITTING

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score
from imblearn.over_sampling import ADASYN
import warnings
from sklearn.exceptions import UndefinedMetricWarning
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

warnings.filterwarnings('ignore', category=UndefinedMetricWarning)

# Display class distribution
X = smoothed_df[feature_list + ["posx", "posy"]]
y = smoothed_df['label']

# train_df = pd.concat([train_df1, train_df2], ignore_index=True)
# smoothed_train = pd.concat([smoothed_train1, smoothed_train2], ignore_index=True)

# Training distribution:
class_distribution = Counter(smoothed_train['label'])
print("Class distribution in the dataset:")
for label, count in sorted(class_distribution.items()):
    print(f"Label {label}: {count} samples ({count / len(y) * 100:.2f}%)")

# Test distribution:
class_distribution = Counter(smoothed_test['label'])
print("Class distribution in the dataset:")
for label, count in sorted(class_distribution.items()):
    print(f"Label {label}: {count} samples ({count / len(y) * 100:.2f}%)")

# Prepare data for classification
X_train = smoothed_train[feature_list + ["posx", "posy"]]
y_train = smoothed_train['label']

X_validation = smoothed_validation[feature_list + ["posx", "posy"]]
y_validation = smoothed_validation['label']

X_test = smoothed_test[feature_list + ["posx", "posy"]]
y_test = smoothed_test['label']

# Preprocess data  -  No need a scaler in Random FOREST!!!!, if other model, apply after the splitting
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)
# X_train_scaled = X_train
# X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=feature_list + ["posx", "posy"], index=X.index)
# X_test_scaled = X_test
# X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=feature_list + ["posx", "posy"], index=X.index)


# Create the figure and axes: 2 rows (train/test), then 3 histograms
fig, axs = plt.subplots(2, 4, figsize=(22, 14), gridspec_kw={'width_ratios': [3, 1, 1, 1]})

# Set style
sns.set(style="whitegrid")

# Create consistent color map using turbo
unique_labels = sorted(data_train['label'].unique())  # or union of train + test if needed
palette_colors = sns.color_palette("turbo", n_colors=len(unique_labels))
label_color_dict = {label: color for label, color in zip(unique_labels, palette_colors)}

# Plot train
sns.scatterplot(
    data=data_train,
    x='posx',
    y='posy',
    hue='label',
    palette=label_color_dict,  # Use fixed mapping
    ax=axs[0, 0]
)
axs[0, 0].set_title("Train Set Distribution (posx vs posy)")
axs[0, 0].set_xlabel("Position X (posx)")
axs[0, 0].set_ylabel("Position Y (posy)")
axs[0, 0].legend(title='Label', bbox_to_anchor=(1.05, 1), loc='upper left')

# Plot test with same label-color mapping
sns.scatterplot(
    data=data_test,
    x='posx',
    y='posy',
    hue='label',
    palette=label_color_dict,  # Same dictionary ensures color consistency
    ax=axs[1, 0]
)
axs[1, 0].set_title("Test Set Distribution (posx vs posy)")
axs[1, 0].set_xlabel("Position X (posx)")
axs[1, 0].set_ylabel("Position Y (posy)")
axs[1, 0].legend(title='Label', bbox_to_anchor=(1.05, 1), loc='upper left')

# --- Histograms / Class Distribution Plots ---
plot_class_distribution(smoothed_df['label'], axs[0, 1], "Class Dist. (Entire)")
plot_class_distribution(y_train, axs[0, 2], "Class Dist. (Train)")
plot_class_distribution(y_test, axs[0, 3], "Class Dist. (Test)")

# Hide the unused bottom row histograms
for ax in axs[1, 1:]:
    ax.axis('off')

plt.tight_layout()
plt.show()

# # Drop posx and posy columns from the training and testing sets after plotting
# X_train = X_train.drop(columns=['posx', 'posy'])
# X_test = X_test.drop(columns=['posx', 'posy'])

### Binary Classifier

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler_train1 = StandardScaler()
scaler_train2 = StandardScaler()
scaler_test = StandardScaler()

X_train_selected = X_train[feature_list]
X_validation_selected = X_validation[feature_list]
X_test_selected = X_test[feature_list]

# X_train_selected_1 = X_train_selected[X_train["posy"] <= 140]
# X_train_selected_2 = X_train_selected[X_train["posy"] > 140]

X_train_scaled = scaler_train1.fit_transform(X_train_selected)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_list, index=X_train_selected.index)

X_validation_scaled = scaler_train1.fit_transform(X_validation_selected)
X_validation_scaled = pd.DataFrame(X_validation_scaled, columns=feature_list, index=X_validation_selected.index)

# X_train_scaled2 = scaler_train2.fit_transform(X_train_selected_2)

# X_train_scaled1 = pd.DataFrame(X_train_scaled1, columns=feature_list, index=X_train_selected_1.index)
# X_train_scaled2 = pd.DataFrame(X_train_scaled2, columns=feature_list, index=X_train_selected_2.index)
# X_train_scaled = pd.concat([X_train_scaled1, X_train_scaled2]).sort_index()

X_test_scaled = scaler_test.fit_transform(X_test_selected)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_list, index=X_test_selected.index)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from imblearn.over_sampling import ADASYN
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, balanced_accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE

# Create dataset with selected features (excluding posx, posy)
# X_train_selected = X_train[feature_list]
# X_test_selected = X_test[feature_list]

X_train_selected = X_train_scaled
X_test_selected = X_test_scaled

# Convert labels to binary classification (0 vs all)
y_train_binary = np.where(y_train == 0, 0, 1)
y_test_binary = np.where(y_test == 0, 0, 1)

# Apply ADASYN for class imbalance handling
adasyn = ADASYN(sampling_strategy='auto', random_state=42, n_neighbors=10)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train_selected, y_train_binary)
# X_train_adasyn, y_train_multi = X_train_selected, y_train

# Display new class distribution after ADASYN
adasyn_class_distribution = Counter(y_train_adasyn)
print("\nClass distribution after ADASYN:")
for label, count in sorted(adasyn_class_distribution.items()):
    print(f"Label {label}: {count} samples ({count / len(y_train_adasyn) * 100:.2f}%)")

# Train the Random Forest model
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=5,
    bootstrap=True,
    criterion='entropy',
    class_weight=None,
    random_state=42,
    max_features=0.5,
    n_jobs=-1,
)

# Fit the model on binary data
rf_model.fit(X_train_adasyn, y_train_adasyn)

# Make predictions on training data
y_train_pred = rf_model.predict(X_train_adasyn)

# Make predictions on test data
y_test_pred = rf_model.predict(X_test_selected)

# Evaluate the model on test data
print("\nClassification Report (Test Data):")
print(classification_report(y_test_binary, y_test_pred, zero_division=0))

# Calculate performance metrics
test_accuracy = accuracy_score(y_test_binary, y_test_pred)
test_balanced_acc = balanced_accuracy_score(y_test_binary, y_test_pred)
train_accuracy = accuracy_score(y_train_adasyn, y_train_pred)
train_balanced_acc = balanced_accuracy_score(y_train_adasyn, y_train_pred)

print(f"\nTrain Accuracy: {train_accuracy:.3f}, Train Balanced Accuracy: {train_balanced_acc:.3f}")
print(f"Test Accuracy: {test_accuracy:.3f}, Test Balanced Accuracy: {test_balanced_acc:.3f}")


# Function to create and plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, title, ax):
    cm = confusion_matrix(y_true, y_pred)
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    cm_norm = np.nan_to_num(cm_norm)  # Replace NaN with 0

    sns.heatmap(cm_norm, annot=True, fmt='.3f', cmap='Blues', xticklabels=[0, 1], yticklabels=[0, 1], ax=ax)
    ax.set_title(title)
    ax.set_xlabel('Predicted Label')
    ax.set_ylabel('True Label')


# Create a figure with two subplots for the confusion matrices
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))

# Plot confusion matrices for training and test data
plot_confusion_matrix(y_train_adasyn, y_train_pred, 'Training Data Confusion Matrix', ax1)
plot_confusion_matrix(y_test_binary, y_test_pred, 'Test Data Confusion Matrix', ax2)

plt.tight_layout()
plt.show()


In [ ]:
from binary_classification_adjustment import *

X_test_new = adjust_test_features(X_train, y_train, X_test, y_test_pred)[feature_list]

In [ ]:
from plotter import plot_features

X_test_new = X_test
Xa = X_train_selected.copy()
Xa["label"] = y_train_binary
Xa["posx"] = X_train["posx"]
Xa["posy"] = X_train["posy"]

Xb = X_test_selected.copy()
Xb["label"] = y_test_binary
Xb["posx"] = X_test["posx"]
Xb["posy"] = X_test["posy"]

X_new_test = Xb.copy()
X_new_test["label"] = y_test

X_new_train = Xa.copy()
X_new_train["label"] = y_train

plot_features(Xa, Xb, feature_list)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extract posx and posy from X_test (ensure they are NumPy arrays)
posx = np.array(X_test['posx'])
posy = np.array(X_test['posy'])

# Ensure y_test and y_test_pred are NumPy arrays
y_test_pred = np.array(y_test_pred)

# Identify misclassified points
misclassified = y_test_binary != y_test_pred

# Create figure with two vertically stacked subplots
fig, axes = plt.subplots(ncols=2, figsize=(16, 8), sharex=True, sharey=True)

# Use 'plasma' colormap for better contrast
cmap = 'turbo'

# Initialize lists to store legend handles
scatter_plots = []

# First subplot: Color by true labels
sc = axes[0].scatter(posx[~misclassified], posy[~misclassified],
                     c=y_test_binary[~misclassified], cmap=cmap, marker='o', label='Correctly Classified')

sc_misclassified = axes[0].scatter(posx[misclassified], posy[misclassified],
                                   c=y_test_binary[misclassified], cmap=cmap, marker='x',
                                   linewidth=1, label='Misclassified')

axes[0].set_title('True Labels with Misclassified Points')
axes[0].set_ylabel('posy')

# Second subplot: Color by predicted labels
sc_pred = axes[1].scatter(posx[~misclassified], posy[~misclassified],
                          c=y_test_pred[~misclassified], cmap=cmap, marker='o', label='Correctly Classified')

sc_pred_misclassified = axes[1].scatter(posx[misclassified], posy[misclassified],
                                        c=y_test_pred[misclassified], cmap=cmap, marker='x',
                                        linewidth=1, label='Misclassified')

axes[1].set_title('Predicted Labels with Misclassified Points')
axes[1].set_xlabel('posx')
axes[1].set_ylabel('posy')

# Collect legend handles and labels
handles = [sc, sc_misclassified]
labels = ['Correctly Classified', 'Misclassified']

# Create one global legend outside the plot
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.05), ncol=2, fontsize=12)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
hard_spots_test_index = y_test_pred == 1
hard_spots_test = X_test[hard_spots_test_index]
y_test_hard = y_test[hard_spots_test_index]

# y_train_pred = rf_model.predict(X_train[feature_list])
hard_spots_train_index = y_train != 0
hard_spots_train = X_train[hard_spots_train_index]
y_train_hard = y_train[hard_spots_train_index]

print(X_train.shape)
hard_spots_train.shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from sklearn.cluster import DBSCAN


def reverse_weighted_smooth(df, features_to_smooth, bandwidth=4.0, center_emphasis=3.0):
    # Create a copy of the dataframe to store smoothed values
    df_smoothed = df.copy()

    # Extract spatial coordinates
    coords = df[['posx', 'posy']].values

    # Use DBSCAN to identify clusters
    clustering = DBSCAN(eps=2.0, min_samples=10).fit(coords)
    labels = clustering.labels_

    # Find cluster centers
    centers = []
    center_indices = []

    for label in np.unique(labels):
        if label == -1:  # Skip noise points
            continue

        # Get points in this cluster
        cluster_mask = labels == label
        cluster_points = coords[cluster_mask]

        # Calculate centroid
        centroid = np.mean(cluster_points, axis=0)
        centers.append(centroid)

        # Find the point closest to the centroid
        distances_to_centroid = np.sum((cluster_points - centroid) ** 2, axis=1)
        closest_point_idx = np.where(cluster_mask)[0][np.argmin(distances_to_centroid)]
        center_indices.append(closest_point_idx)

    # Apply smoothing to each feature
    for feature in features_to_smooth:
        if feature in df.columns and feature not in ['posx', 'posy', 'label']:
            # Get feature values as array
            feature_values = df[feature].values.copy()
            original_values = feature_values.copy()

            # For each point, calculate weighted average based on distance
            for i in range(len(coords)):
                # Calculate distances to all other points
                dists = np.sqrt(np.sum((coords - coords[i]) ** 2, axis=1))

                # Create weights with Gaussian kernel
                kernel_weights = np.exp(-0.5 * (dists / bandwidth) ** 2)

                # Determine if this point is a center
                is_center = i in center_indices

                if is_center:
                    # For center points, preserve their original value or even emphasize it
                    central_value = original_values[i]
                    # Blend with weighted average, but give more weight to the original center value
                    avg_value = np.sum(kernel_weights * original_values) / np.sum(kernel_weights)
                    feature_values[i] = (central_value * center_emphasis + avg_value) / (center_emphasis + 1)
                else:
                    # Regular smoothing for non-center points
                    feature_values[i] = np.sum(kernel_weights * original_values) / np.sum(kernel_weights)

            # Update the smoothed dataframe
            df_smoothed[feature] = feature_values

    return df_smoothed, centers, center_indices, labels


def visualize_clusters_and_centers(df, center_indices, labels, ax, title):
    # Create a colormap for clusters
    unique_labels = np.unique(labels)
    colors = plt.cm.tab20(np.linspace(0, 1, len(unique_labels)))

    # Plot points colored by cluster
    for i, label in enumerate(unique_labels):
        if label == -1:
            # Black for noise points
            cluster_color = 'black'
        else:
            cluster_color = colors[i]

        mask = labels == label
        ax.scatter(df.loc[mask, 'posx'], df.loc[mask, 'posy'],
                   c=[cluster_color], edgecolor='k', s=50,
                   alpha=0.6)

    # Plot centers with X markers
    center_points = df.iloc[center_indices][['posx', 'posy']].values
    ax.scatter(center_points[:, 0], center_points[:, 1],
               c='red', marker='X', s=200, edgecolor='k')

    ax.set_title(f'{title} with Cluster Centers')
    ax.set_xlabel('posx')
    ax.set_ylabel('posy')


features_to_smooth = [f for f in feature_list if f not in ['posx', 'posy']]
hard_spots_test_smoothed, centers, center_indices, labels = reverse_weighted_smooth(
    hard_spots_test, features_to_smooth, bandwidth=3.0, center_emphasis=0.00001)

hard_spots_train_smoothed, centers2, center_indices2, labels2 = reverse_weighted_smooth(
    hard_spots_train, features_to_smooth, bandwidth=3.0, center_emphasis=0.00001)

# Visualize the clusters and centers7
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
visualize_clusters_and_centers(hard_spots_test, center_indices, labels, axes[1], "test")
visualize_clusters_and_centers(hard_spots_train, center_indices2, labels2, axes[0], "train")


# Plot the smoothing effect for comparison
def plot_smoothing_effect(test_original, test_smooth, train_original, train_smoothed, feature):
    fig, axes = plt.subplots(2, 2, figsize=(8, 8))

    # Plot original values
    sc1 = axes[1][0].scatter(test_original['posx'], test_original['posy'],
                             c=test_original[feature], cmap='viridis',
                             s=50, alpha=0.8)
    axes[1][0].set_title(f"Test: Original {feature}")
    axes[1][0].set_xlabel("posx")
    axes[1][0].set_ylabel("posy")

    sc1 = axes[0][0].scatter(train_original['posx'], train_original['posy'],
                             c=train_original[feature], cmap='viridis',
                             s=50, alpha=0.8)
    axes[0][0].set_title(f"Train: Original {feature}")
    axes[0][0].set_xlabel("posx")
    axes[0][0].set_ylabel("posy")

    # Plot smoothed values
    sc2 = axes[1][1].scatter(test_smooth['posx'], test_smooth['posy'],
                             c=test_smooth[feature], cmap='viridis',
                             s=50, alpha=0.8)
    axes[1][1].set_title(f"Test: Smoothed {feature}")
    axes[1][1].set_xlabel("posx")
    axes[1][1].set_ylabel("posy")

    sc2 = axes[0][1].scatter(train_smoothed['posx'], train_smoothed['posy'],
                             c=train_smoothed[feature], cmap='viridis',
                             s=50, alpha=0.8)
    axes[0][1].set_title(f"Train: Smoothed {feature}")
    axes[0][1].set_xlabel("posx")
    axes[0][1].set_ylabel("posy")

    plt.tight_layout()
    plt.show()


for feature in features_to_smooth:
    plot_smoothing_effect(hard_spots_test, hard_spots_test_smoothed, hard_spots_train, hard_spots_train_smoothed,
                          feature)

### Full Classifier

In [ ]:
from plotter import plot_confusion_matrix

test_index = y_test_pred == 1
X_test_new_nz = X_test_new[test_index]
y_test_nz = y_test[test_index]

train_index = y_train != 0
X_train_nz = X_train[train_index]
y_train_nz = y_train[train_index]

X_train_selected = X_train_nz[feature_list]
X_test_selected = X_test_new_nz[feature_list]  #X_test  --- X_test_new

# X_train_selected = hard_spots_train_smoothed
# X_test_selected = hard_spots_test_smoothed
# y_train = y_train_hard
# y_test = y_test_hard

# Apply ADASYN for adaptive oversampling
# adasyn = ADASYN(sampling_strategy='auto', random_state=42, n_neighbors=5)
# X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train_selected, y_train)

X_train_adasyn, y_train_adasyn = X_train_selected, y_train_nz

# Display new class distribution after ADASYN
adasyn_class_distribution = Counter(y_train_adasyn)
print("\nClass distribution after ADASYN:")
for label, count in sorted(adasyn_class_distribution.items()):
    print(f"Label {label}: {count} samples ({count / len(y_train_adasyn) * 100:.2f}%)")

# Train the Random Forest model
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=5,
    bootstrap=True,
    criterion='entropy',
    class_weight=None,
    random_state=42,
    max_features=0.5,
    n_jobs=-1,
)

# Fit the model
rf_model.fit(X_train_adasyn, y_train_adasyn)

# Make predictions on training data
y_train_pred = rf_model.predict(X_train_adasyn)

# Make predictions on test data
y_test_pred_nz = rf_model.predict(X_test_selected)

# Evaluate the model on test data
print("\nClassification Report (Test Data):")
print(classification_report(y_test_nz, y_test_pred_nz, zero_division=0))

# Calculate performance metrics
test_accuracy = accuracy_score(y_test_nz, y_test_pred_nz)
test_balanced_acc = balanced_accuracy_score(y_test_nz, y_test_pred_nz)
train_accuracy = accuracy_score(y_train_adasyn, y_train_pred)
train_balanced_acc = balanced_accuracy_score(y_train_adasyn, y_train_pred)

# Create a figure with two subplots for the confusion matrices
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))

# Plot confusion matrices for training and test data
plot_confusion_matrix(y_train_adasyn, y_train_pred, 'Training Data Confusion Matrix', ax1)
plot_confusion_matrix(y_test_nz, y_test_pred_nz, 'Test Data Confusion Matrix', ax2)

plt.tight_layout()
plt.show()


In [ ]:
import shap

explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(X_test)

# For binary classification, shap_values[1] is for class 1
shap.summary_plot(shap_values[1], X_test)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extract posx and posy from X_test (ensure they are NumPy arrays)
posx = np.array(X_test['posx'])
posy = np.array(X_test['posy'])

# Ensure y_test and y_test_pred are NumPy arrays
y_test = np.array(y_test)
y_test_pred = np.array(y_test_pred)

# Identify misclassified points
misclassified = y_test != y_test_pred

# Create figure with two vertically stacked subplots
fig, axes = plt.subplots(ncols=2, figsize=(16, 8), sharex=True, sharey=True)

# Use 'plasma' colormap for better contrast
cmap = 'turbo'

# Initialize lists to store legend handles
scatter_plots = []

# First subplot: Color by true labels
sc = axes[0].scatter(posx[~misclassified], posy[~misclassified],
                     c=y_test[~misclassified], cmap=cmap, marker='o', label='Correctly Classified')

sc_misclassified = axes[0].scatter(posx[misclassified], posy[misclassified],
                                   c=y_test[misclassified], cmap=cmap, marker='x',
                                   linewidth=1, label='Misclassified')

axes[0].set_title('True Labels with Misclassified Points')
axes[0].set_ylabel('posy')

# Second subplot: Color by predicted labels
sc_pred = axes[1].scatter(posx[~misclassified], posy[~misclassified],
                          c=y_test_pred[~misclassified], cmap=cmap, marker='o', label='Correctly Classified')

sc_pred_misclassified = axes[1].scatter(posx[misclassified], posy[misclassified],
                                        c=y_test_pred[misclassified], cmap=cmap, marker='x',
                                        linewidth=1, label='Misclassified')

axes[1].set_title('Predicted Labels with Misclassified Points')
axes[1].set_xlabel('posx')
axes[1].set_ylabel('posy')

# Collect legend handles and labels
handles = [sc, sc_misclassified]
labels = ['Correctly Classified', 'Misclassified']

# Create one global legend outside the plot
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.05), ncol=2, fontsize=12)

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
from scipy.stats import wasserstein_distance
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif

# --- Random Forest Feature Importances ---

# Assuming rf_model is your trained model and top_features is your list of features

feature_importances = rf_model.feature_importances_

# Create a DataFrame for feature importances and sort them
importance_df = pd.DataFrame({
    'feature': feature_list,
    'importance': feature_importances
}).sort_values('importance', ascending=True)

# --- Mutual Information Scores ---

X = data_train[feature_list]
y = data_train['label']

# Compute mutual information scores for each feature
mi_scores = mutual_info_classif(X, y, random_state=42)

X2 = smoothed_train[feature_list]
y2 = smoothed_train['label']

# Compute mutual information scores for each feature (smoothed)
mi_scores2 = mutual_info_classif(X2, y2, random_state=42)

# Create a DataFrame for mutual information scores and sort them


# --- Mutual Information on Test Set ---
X_test_mi = smoothed_test[feature_list]
y_test_mi = smoothed_test['label']
mi_scores_test = mutual_info_classif(X_test_mi, y_test_mi, random_state=42)

X_validation_mi = smoothed_validation[feature_list]
y_validation_mi = smoothed_validation['label']
mi_scores_validation = mutual_info_classif(X_validation_mi, y_validation_mi, random_state=42)

# Combine all MI scores
mi_df = pd.DataFrame({
    'feature': feature_list,
    'original': mi_scores,
    'smoothed': mi_scores2,
    'test': mi_scores_test,
    'validation': mi_scores_validation
}).sort_values('original', ascending=True)

# Select only label 3, 4, 5 from train and test
selected_labels = [3, 4, 5]
train_mask = y_train.isin(selected_labels)
test_mask = y_test.isin(selected_labels)

X_train_filtered = X_train_selected[train_mask]
X_test_filtered = X_test_selected[test_mask]

# --- Compute Wasserstein distance for each feature ---
distance_data = {
    'feature': [],
    'wasserstein_distance': []
}

for feat in feature_list:
    train_feat = X_train_filtered[feat]
    test_feat = X_test_filtered[feat]
    distance = wasserstein_distance(train_feat, test_feat)
    distance_data['feature'].append(feat)
    distance_data['wasserstein_distance'].append(distance)

# distance_df = pd.DataFrame(distance_data).sort_values('wasserstein_distance', ascending=True)
distance_df = pd.DataFrame(distance_data)

# 3 subplots: Importance | Mutual Info | Distance
fig, axes = plt.subplots(1, 3, figsize=(24, 9), sharey=True)

# 1. Random Forest Feature Importances
sns.barplot(x='importance', y='feature', data=importance_df, palette='viridis', ax=axes[0], hue="feature", legend=False)
axes[0].set_title('Random Forest Feature Importances', fontsize=16)
axes[0].set_xlabel('Importance', fontsize=12)
axes[0].set_ylabel('Features', fontsize=12)

# 2. Mutual Information: Original, Smoothed, Test
bar_width = 0.45
y_positions = np.arange(len(mi_df))
ax = axes[1]

ax.barh(y_positions, mi_df['original'], height=bar_width, color='red', alpha=0.6, label='Original')
ax.barh(y_positions, mi_df['smoothed'], height=bar_width, color='blue', alpha=0.6, label='Smoothed')
ax.barh(y_positions + bar_width, mi_df['test'], height=bar_width, color='green', alpha=0.6, label='Test')

ax.set_yticks(y_positions)
ax.set_yticklabels(mi_df['feature'])
ax.set_title('Mutual Information Scores', fontsize=16)
ax.set_xlabel('Mutual Info Score', fontsize=12)
ax.set_ylabel('')
ax.legend()

# 3. Train/Test Distribution Distance
sns.barplot(x='wasserstein_distance', y='feature', data=distance_df, palette='mako', ax=axes[2], hue='feature',
            legend=False)
axes[2].set_title('Train vs Test Wasserstein Distance', fontsize=16)
axes[2].set_xlabel('Distance', fontsize=12)
axes[2].set_ylabel('')
axes[2].set_xlim(0, 1)

plt.tight_layout()
plt.show()

# Combine everything into a single DataFrame for easy filtering
combined_df = mi_df.merge(distance_df, on='feature')

# Filter by the given conditions
selected_features = combined_df[
    # (combined_df['smoothed'] > 0.4) &
    # (combined_df['test'] > 0.4) &
    (combined_df['wasserstein_distance'] < 0.12)
]

# Output the feature names
print("✅ Features with MI (smoothed train & test > 0.4) and Wasserstein < 0.05:")
print(selected_features['feature'].tolist())

In [ ]:
from plotter import plot_features

hard_spots_test["label"] = y_test_hard
hard_spots_train["label"] = y_train_hard

hard_spots_test_smoothed = hard_spots_test_smoothed.copy()
hard_spots_test_smoothed["label"] = y_test_hard

hard_spots_train_smoothed = hard_spots_train_smoothed.copy()
hard_spots_train_smoothed["label"] = y_train_hard

# Call function with datasets
plot_features(hard_spots_train_smoothed, hard_spots_test_smoothed, feature_list)
print("\n\n")
X_train_nz = X_train_nz.copy()
X_train_nz["label"] = y_train_nz

X_test_new_nz = X_test_new_nz.copy()
X_test_new_nz["label"] = y_test_nz

plot_features(X_train_nz, X_test_new_nz, feature_list)


In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Define models in a list of tuples (name, model instance)
models = [
    ('SVM', SVC(kernel='rbf', C=0.5, gamma='scale', probability=True, random_state=42)),  # n_jobs not supported
    ('KNN', KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2, n_jobs=-1)),
    ('Logistic Regression', LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1)),
    ('Random Forest', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)),
    ('Gradient Boosting', GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)),
    # no n_jobs
    ('Naive Bayes', GaussianNB()),  # no n_jobs
    ('Decision Tree', DecisionTreeClassifier(random_state=42)),  # no n_jobs
    ('MLP Neural Net', MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)),  # no n_jobs
]


def fit_and_evaluate_models(models, X_train, y_train, X_test, y_test):
    results = []

    for i, (name, model) in enumerate(models):
        print(f"\nTraining {name}...")
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        # Evaluation metrics
        train_acc = accuracy_score(y_train, y_train_pred)
        test_acc = accuracy_score(y_test, y_test_pred)
        train_bal_acc = balanced_accuracy_score(y_train, y_train_pred)
        test_bal_acc = balanced_accuracy_score(y_test, y_test_pred)

        print(f"{name} Training Accuracy: {train_acc:.4f}")
        print(f"{name} Training Balanced Accuracy: {train_bal_acc:.4f}")
        print(f"{name} Test Accuracy: {test_acc:.4f}")
        print(f"{name} Test Balanced Accuracy: {test_bal_acc:.4f}")

        results.append({
            'Model': name,
            'Train Accuracy': train_acc,
            'Train Balanced Accuracy': train_bal_acc,
            'Test Accuracy': test_acc,
            'Test Balanced Accuracy': test_bal_acc
        })

        # Plot for this model
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        plot_confusion_matrix(y_train, y_train_pred, f'{name} - Train', axes[0])
        plot_confusion_matrix(y_test, y_test_pred, f'{name} - Test', axes[1])
        plt.tight_layout()
        plt.show()

    return results


# Example call
results = fit_and_evaluate_models(models, X_train_adasyn, y_train_adasyn, X_test_selected, y_test)


# Plot comparison bar chart
def plot_model_comparison(results):
    import pandas as pd
    import seaborn as sns

    df = pd.DataFrame(results)
    df_melted = df.melt(id_vars='Model', var_name='Metric', value_name='Score')

    plt.figure(figsize=(12, 6))
    sns.barplot(data=df_melted, x='Model', y='Score', hue='Metric')
    plt.title('Model Comparison')
    plt.ylim(0, 1.05)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()


plot_model_comparison(results)


In [ ]:
# Standard scikit-learn imports
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    ExtraTreesClassifier,
    VotingClassifier,
    StackingClassifier
)
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

models = [
    # Ensemble methods that handle imbalance well
    ('XGBoost', XGBClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=6,
        min_child_weight=2,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    )),

    # LightGBM - fast and handles complex relationships well
    ('LightGBM', LGBMClassifier(
        n_estimators=200,
        learning_rate=0.05,
        num_leaves=31,
        feature_fraction=0.8,
        bagging_fraction=0.8,
        bagging_freq=5,
        random_state=42,
        n_jobs=-1
    )),

    # Voting classifier combining your best models
    ('Voting Classifier', VotingClassifier(
        estimators=[
            ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
            ('gb', GradientBoostingClassifier(n_estimators=100, learning_rate=0.1)),
            ('svm', SVC(probability=True, kernel='rbf'))
        ],
        voting='soft'
    )),

    # CatBoost - handles categorical features well and often outperforms
    ('CatBoost', CatBoostClassifier(
        iterations=200,
        learning_rate=0.05,
        depth=6,
        l2_leaf_reg=3,
        random_state=42,
        verbose=0
    )),

    # Extremely Randomized Trees - reduces variance compared to RF
    ('Extra Trees', ExtraTreesClassifier(
        n_estimators=100,
        random_state=42,
        n_jobs=-1
    )),

    # Deep MLP with dropout for better generalization
    ('Deep Neural Net', MLPClassifier(
        hidden_layer_sizes=(100, 50, 25),
        activation='relu',
        solver='adam',
        alpha=0.0001,
        batch_size='auto',
        learning_rate='adaptive',
        max_iter=500,
        early_stopping=True,
        validation_fraction=0.2,
        random_state=42
    ))
]

results = fit_and_evaluate_models(models, X_train_adasyn, y_train_adasyn, X_test_selected, y_test)
plot_model_comparison(results)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

class_weights = {0: 1, 1: 1, 2: 1, 3: 3, 4: 3, 5: 3}

# Apply to any model that supports class_weight
weighted_rf = RandomForestClassifier(
    n_estimators=200,
    class_weight=class_weights,
    random_state=42,
    n_jobs=-1
)

pca_svm = Pipeline([
    ('pca', PCA(n_components=15, random_state=42)),
    ('svm', SVC(kernel='rbf', probability=True))
])

tsne_rf = Pipeline([
    ('tsne', TSNE(n_components=3, random_state=42, perplexity=30, n_jobs=-1)),
    ('rf', RandomForestClassifier(n_estimators=100, n_jobs=-1))
])

models = [('RF', weighted_rf), ('PCA', pca_svm), ('TSNE', tsne_rf)]

results = fit_and_evaluate_models(models, X_train_adasyn, y_train_adasyn, X_test_selected, y_test)
plot_model_comparison(results)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pandas as pd

# Apply t-SNE on the test set (2D)
tsne = TSNE(n_components=2, random_state=42, perplexity=10, n_jobs=-1)
X_test_tsne = tsne.fit_transform(X_test_selected)

# Plot t-SNE results with actual test labels
plt.figure(figsize=(10, 6))
scatter = plt.scatter(X_test_tsne[:, 0], X_test_tsne[:, 1], c=y_test, cmap='tab10', alpha=0.7)
plt.colorbar(scatter, ticks=range(len(set(y_test))))
plt.title("t-SNE projection (2D) of Test Set")
plt.xlabel("TSNE-1")
plt.ylabel("TSNE-2")
plt.grid(True)
plt.show()

# Optionally: print as DataFrame
df_tsne = pd.DataFrame({
    'TSNE-1': X_test_tsne[:, 0],
    'TSNE-2': X_test_tsne[:, 1],
    'Label': y_test
})
print(df_tsne.head())


### Features Extraction

In [ ]:
import pandas as pd
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute

dfs = {idx: pd.DataFrame({'time': row['t'], 'Fz_s': row['Fz_s']})
       for idx, row in data_train.iterrows()}

long_df = pd.concat([df.assign(id=idx) for idx, df in dfs.items()], ignore_index=True)

# Step 1: Extract features from the time series data.
extracted_features = extract_features(long_df, column_id="id", column_sort="time", column_value="Fz_s")

# Step 2: Impute any missing values that might have resulted from the extraction process.
imputed_features = impute(extracted_features)

# Step 3: Select features that significantly correlate with the labels.
relevant_features = select_features(imputed_features, data_train["label"], fdr_level=0.0000001)  # Stricter selection

In [ ]:
import pandas as pd
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute

# Create a dictionary of DataFrames for each row in smoothed_train

# data_train_features = data_train[(data_train['posy'] > 99) & (data_train['posy'] < 101)]
data_train_features = data_train

dfs = {
    idx: pd.DataFrame({
        'time': row['Timei'][::4],
        'Fz_s': row['Fi'][::4],
        'Pz_s': row['Pi'][::4],
    }) for idx, row in data_train_features.iterrows()
}

# Convert to long format by stacking Fz_s and Pz_s under a new 'kind' column
long_df = pd.concat(
    [df.assign(id=idx).melt(id_vars=['time', 'id'], var_name='kind', value_name='value') for idx, df in dfs.items()],
    ignore_index=True
)

long_df

In [ ]:
from tsfresh.feature_extraction import MinimalFCParameters, EfficientFCParameters

# Extract features from both Fz_s and Pz_s
extracted_features = extract_features(
    long_df,
    column_id="id",
    column_sort="time",
    column_kind="kind",
    column_value="value",
    n_jobs=8,
    default_fc_parameters=EfficientFCParameters()
)

# Impute missing values
imputed_features = impute(extracted_features)

# Select relevant features
relevant_features = select_features(imputed_features, data_train_features["label"], fdr_level=0.05)


In [ ]:
from sklearn.feature_selection import mutual_info_classif

# Compute mutual information scores
mi_scores = mutual_info_classif(imputed_features, data_train["label"])

# Create DataFrame with feature importance
feature_importance = pd.DataFrame({"feature": imputed_features.columns, "importance": mi_scores})

# Sort features by importance (highest first)
top_20_features = feature_importance.sort_values(by="importance", ascending=False).head(20)

# Extract the corresponding feature columns from the dataset
top_20_feature_names = top_20_features["feature"].tolist()
top_20_selected = imputed_features[top_20_feature_names]

print(f"Top 20 Most Informative Features:\n{top_20_selected.columns}")


In [ ]:
from tsfresh.feature_extraction import extract_features, EfficientFCParameters
from tsfresh.utilities.dataframe_functions import impute

dfs = {
    idx: pd.DataFrame({
        'time': row['Timei'][::4],
        'Fz_s': row['Fi'][::4],
        'Pz_s': row['Pi'][::4],
    }) for idx, row in data_test.iterrows()
}

# Convert to long format by stacking Fz_s and Pz_s under a new 'kind' column
long_df_test = pd.concat(
    [df.assign(id=idx).melt(id_vars=['time', 'id'], var_name='kind', value_name='value') for idx, df in dfs.items()],
    ignore_index=True
)

# 1. Extract all features on test set (same settings as train)
test_features = extract_features(
    long_df_test,
    column_id="id",
    column_sort="time",
    column_kind="kind",
    column_value="value",
    n_jobs=8,
    default_fc_parameters=EfficientFCParameters()
)

# 2. Impute missing values
test_features_imputed = impute(test_features)

# 3. Select only your top‑20 feature columns
test_top20 = test_features_imputed[top_20_feature_names].copy()

# 4. (Optional) verify
print(f"Computed top‑20 features on test set:\n{test_top20.columns.tolist()}")
print(f"Shape: {test_top20.shape}")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Train/Test Split
# X_train, X_test, y_train, y_test = train_test_split(
#     top_20_selected,
#     data_train_features["label"],
#     test_size=0.2,
#     random_state=42
# )

X_train, X_test, y_train, y_test = top_20_selected, test_top20, data_train_features["label"], data_test["label"]

# Train a Random Forest Classifier
clf = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    class_weight="balanced"
)
clf.fit(X_train, y_train)

# Predict
y_pred = clf.predict(X_test)

# 1) Overall accuracy in percent
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc * 100:.2f}%")

# 2) Raw confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=sorted(set(y_test)))

# 3) Normalize to percentages
#    - overall: divide by total samples
cm_pct_total = cm / cm.sum() * 100
#    - or per‐class (rows): divide each row by its sum
cm_pct_rows = cm.astype(float)
row_sums = cm_pct_rows.sum(axis=1)[:, np.newaxis]
cm_pct_rows = (cm_pct_rows / row_sums) * 100

# 4) Plot normalized (row‐wise) confusion matrix as percentages
plt.figure(figsize=(6, 5))
sns.heatmap(
    cm_pct_rows,
    annot=True,
    fmt=".1f",
    cmap="Blues",
    xticklabels=sorted(set(y_test)),
    yticklabels=sorted(set(y_test))
)
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("Confusion Matrix (per class %)")
plt.show()
